In [4]:
import numpy
import scipy.special

In [17]:
'''
代码框架：
'''
class  NeuralNetWork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #初始化网络，设置输入层，中间层，和输出层节点数
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        #设置学习率
        self.lr = learningrate
        '''
        初始化权重矩阵。
        由于权重不一定都是正的，它完全可以是负数，因此我们在初始化时，
        把所有权重初始化为-0.5到0.5之间
        初始化权重矩阵，我们有两个权重矩阵，一个是wih表示输入层和中间层节点间链路权重形成的矩阵
        一个是who,表示中间层和输出层间链路权重形成的矩阵
        '''
        self.wih = numpy.random.rand(self.hnodes, self.inodes) - 0.5
        self.who = numpy.random.rand(self.onodes, self.hnodes) - 0.5
        '''
        hidden是个一维向量，每个元素对应着中间层某个节点从上一层神经元传过来后的信号量
        总和.于是每个节点就得执行激活函数，得到的结果将作为信号输出到下一层.
        sigmod函数在Python中可以直接调用，我们要做的就是准备好参数。
        我们先把这个函数在初始化函数中设定好，
        scipy.special.expit对应的是sigmod函数.
        lambda是Python关键字，类似C语言中的宏定义.
        我们调用self.activation_function(x)时，编译器会把其转换为spicy.special_expit(x)。
        '''
        self.activation_function = lambda x:scipy.special.expit(x)
        pass
    def  train(self,inputs_list, targets_list):
        #根据输入的训练数据更新节点链路权重
        '''
        训练过程分两步：
        第一步是计算输入训练数据，给出网络的计算结果，
        这点跟我们前面实现的query()功能很像。
        第二步是将计算结果与正确结果相比对，获取误差，
        采用误差反向传播法更新网络里的每条链路权重。
        我们先用代码完成第一步.

        inputs_list:输入的训练数据;
        targets_list:训练数据对应的正确结果。
        '''
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        #计算信号经过输入层后产生的信号量
        hidden_inputs = numpy.dot(self.wih, inputs)
        #中间层神经元对输入的信号做激活函数后得到输出信号
        hidden_outputs = self.activation_function(hidden_inputs)
        #输出层接收来自中间层的信号量
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #输出层对信号量进行激活函数后得到最终输出信号
        final_outputs = self.activation_function(final_inputs)
        
        #计算误差
        output_errors = targets - final_outputs
        hidden_errors = numpy.dot(self.who.T, output_errors*final_outputs*(1-final_outputs))
        #根据误差计算链路权重的更新量，然后把更新加到原来链路权重上
        self.who += self.lr * numpy.dot((output_errors * final_outputs *(1 - final_outputs)),
                                       numpy.transpose(hidden_outputs))
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)),
                                       numpy.transpose(inputs))
        pass
        
    def  query(self,inputs):
        '''
        接着我们先看query函数的实现，它接收输入数据，通过神经网络的层层计算后，
        在输出层输出最终结果。
        输入数据要依次经过输入层，中间层，和输出层，
        并且在每层的节点中还得执行激活函数以便形成对下一层节点的输出信号。
        我们知道可以通过矩阵运算把这一系列复杂的运算流程给统一起来。
        '''
        #根据输入数据计算并输出答案
        hidden_inputs = numpy.dot(self.wih, inputs)
        '''
        由此我们就可以分别调用激活函数计算中间层的输出信号，
        以及输出层经过激活函数后形成的输出信号，
        '''
        #计算中间层经过激活函数后形成的输出信号量
        hidden_outputs = self.activation_function(hidden_inputs)
        #计算最外层接收到的信号量
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #计算最外层神经元经过激活函数后输出的信号量
        final_outputs = self.activation_function(final_inputs)
        print(final_outputs)
        return final_outputs

In [18]:
#初始化网络
'''
由于一张图片总共有28*28 = 784个数值，因此我们需要让网络的输入层具备784个输入节点
'''
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
learning_rate = 0.1
n = NeuralNetWork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [19]:
#读入训练数据
#open函数里的路径根据数据存储的路径来设定
training_data_file = open("dataset/mnist_train.csv",'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [20]:
#加入epocs,设定网络的训练循环次数
epochs = 5
for e in range(epochs):
    #把数据依靠','区分，并分别读入
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]))/255.0 * 0.99 + 0.01
        #设置图片与数值的对应关系
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)

test_data_file = open("dataset/mnist_test.csv")
test_data_list = test_data_file.readlines()
test_data_file.close()
scores = []
for record in test_data_list:
    all_values = record.split(',')
    correct_number = int(all_values[0])
    print("该图片对应的数字为:",correct_number)
    #预处理数字图片
    inputs = (numpy.asfarray(all_values[1:])) / 255.0 * 0.99 + 0.01
    #让网络判断图片对应的数字
    outputs = n.query(inputs)
    #找到数值最大的神经元对应的编号
    label = numpy.argmax(outputs)
    print("网络认为图片的数字是：", label)
    if label == correct_number:
        scores.append(1)
    else:
        scores.append(0)
print(scores)

#计算图片判断的成功率
scores_array = numpy.asarray(scores)
print("perfermance = ", scores_array.sum() / scores_array.size)

该图片对应的数字为: 7
[0.01223979 0.07251191 0.01253982 0.00968508 0.05239736 0.02042917
 0.09622979 0.03337636 0.3971121  0.08698497]
网络认为图片的数字是： 8
该图片对应的数字为: 2
[0.08998117 0.09517004 0.01622107 0.13185574 0.00164002 0.0481161
 0.12797953 0.07803355 0.03926736 0.01210639]
网络认为图片的数字是： 3
该图片对应的数字为: 1
[0.01597004 0.68756653 0.02194715 0.03817215 0.03612004 0.21318643
 0.02126607 0.16161468 0.38739486 0.06741604]
网络认为图片的数字是： 1
该图片对应的数字为: 0
[0.70793097 0.03595711 0.02011723 0.01272623 0.01689002 0.01694173
 0.22512199 0.02043402 0.08037179 0.00593656]
网络认为图片的数字是： 0
该图片对应的数字为: 4
[0.02320202 0.0024079  0.00383723 0.02342524 0.46528827 0.00542849
 0.01610356 0.194111   0.23030176 0.06421638]
网络认为图片的数字是： 4
该图片对应的数字为: 1
[0.01964161 0.85731024 0.02494267 0.03967228 0.02447575 0.08478108
 0.01833166 0.06077162 0.21940409 0.03150009]
网络认为图片的数字是： 1
该图片对应的数字为: 4
[0.00930243 0.01771445 0.00411676 0.27011503 0.03547126 0.05611645
 0.1840206  0.05103378 0.27193962 0.07195749]
网络认为图片的数字是： 8
该图片对应的数字为: 9
[0.01798